In [1]:
import numpy as np
import time
import pandas as pd
import requests
import seaborn as sns

from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
driver = webdriver.Chrome("driver/chromedriver")
driver.get("https://fbref.com/en/search/search.fcgi?search=")

C:\Users\HP\AppData\Local\Temp\ipykernel_3180\4048836481.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("driver/chromedriver")


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:
def search_player(player_name):
    player_name = player_name.replace('.', '')
    # form 찾기
    f_big = driver.find_element_by_name("f_big")
    # 검색입력 찾기 & 입력
    ac_input = f_big.find_element_by_class_name("ac-input")
    ac_input.clear()
    ac_input.send_keys(player_name)
    # 검색버튼 클릭
    submit_input = f_big.find_element_by_xpath("//input[@type='submit']")
    submit_input.click()
    # 검색 로딩 대기
    time.sleep(3)
    
    ars_keyword = player_name.split(' ')
    match_url = []
    
    content = driver.find_element_by_id("content")

    if "0 hits" not in content.get_attribute("innerHTML"):
        search_players = content.find_elements_by_css_selector("#players > div.search-item")

        for players in search_players:
            match_keyword = 0
            try:
                a_player = players.find_element_by_css_selector("div.search-item-name > a")
            except:
                try:
                    a_player = players.find_element_by_css_selector("div.search-item-name > strong > a")
                except:
                    continue
            
            name = a_player.text.split(' ')

            if len(name) >= len(ars_keyword):
                for i in range(len(ars_keyword)):
                    if ars_keyword[i] in name[i]:
                        match_keyword += 1

            if match_keyword >= len(ars_keyword):
                match_url.append(a_player.get_attribute('href'))
    else:
        print("No Data::", search_keyword)
        
    return match_url

In [138]:
df = pd.read_csv("./FIFA_train.csv")
df.head()

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0


In [139]:
df_name_new = df['name']

In [ ]:
url_cnt = []

for i in range(len(df_name_new)):
    player_url = search_player(df_name_new[i])
    url_cnt.append(len(player_url))
    
df['url_cnt'] = url_cnt

No Data:: L. Messi
